**Tarea 7 - Integracion Taylor**

Estudiantes: Yuriko Yamamoto, Ignacio Vargas

**Ejercicio 1:** Usando su implementación de polinomios de Taylor, escriban un integrador para la ecuación diferencial que se desarrolló en este ejemplo, esto es, con la condición inicial x(0)=3.

El integrador debe hacer las operaciones necesarias para obtener automáticamente los coeficientes x[k], en cada paso de integración, a partir de la condición inicial local. Un requisito básico para esto es que tengan una implementación de la función Pα(x)=[g(x)]α con g(x) un polinomio, que obtuvieron en la "Tarea6", y que funcione bien en particular para alpha::Int.

La implementación debe consistir de varias funciones: 

- Una función donde se calculen los coeficientes $x_{[k]}(t_0)$ de la expansión. Esta función deberá llamar a otra donde se implementan las ecuaciones de movimiento.

- Una función donde se obtenga el paso de integración $h$.

- Otra función donde se haga la suma por el método de Horner.

- Finalmente, una función que use las funciones anteriores para hacer la integración desde un tiempo inicial a uno final. En este punto, *fingiremos ignorancia*, en el sentido que el tiempo inicial es cero, y el tiempo final será $0.5$ (que está más allá de donde hay problemas).


Dado que conocemos la solución analítica, den como función de $t$ el error relativo de su integrador (respecto al valor del resultado analítico).

In [2]:
type Taylor{T<:Number}
    
    coeficientes :: Array{T,1}
    orden :: T
    
    #Constructor interno
    function Taylor(coeficientes::Array{T,1}, orden::T)
        
        longitud_coeficientes= length(coeficientes)
        orden = max(orden, longitud_coeficientes-1)
        
        vector = zeros(T, orden+1)
        
        vector[1:longitud_coeficientes] = coeficientes[1:end]
        new(vector, orden)
        
    end
end

#Constructor externo
Taylor{T<:Number}(x::Taylor{T}, orden::Integer) = Taylor{T}(x.coeficientes, orden)
#Taylor{T<:Number}(x::Taylor{T}) = Taylor{T}(x.coeficientes, x.orden)
Taylor{T<:Number}(x::Taylor{T}) = x
Taylor{T<:Number}(coeficientes::Array{T,1}, orden::Integer) = Taylor{T}(coeficientes, orden)
Taylor{T<:Number}(coeficientes::Array{T,1}) = Taylor{T}(coeficientes, length(coeficientes)-1)
Taylor{T<:Number}(x::T, orden::Integer) = Taylor{T}([x], orden)
Taylor{T<:Number}(x::T) = Taylor{T}([x], 0)

Taylor{T<:Number}

In [3]:
#Primer Ejercicio


#Exponente
function ^(a::Taylor, x::Integer)
    uno = one(a)
    if x < 0
        return uno / a^(-x)
    elseif x == 0
        return uno
    elseif x%2 == 0 # par
        if x == 2
            return square(a)
        else
            pow = div(x, 2)
            return square( a^pow )
        end
    else  # impar
        if x == 1
            return a
        else
            expon = div(x-1, 2)
            return a*square( a^expon )
        end
    end
end


^ (generic function with 46 methods)

In [5]:
f(x)=x^2

f (generic function with 1 method)

In [7]:
x_0 = 3

3

**Ejercicio 2:** Repitan los cálculos usando el método de Runge-Kutta de 4o orden con paso de integración fijo (que es lo más sofisticado que conocen hasta ahora) y comparen los resultados del error relativo con el método de Taylor. En particular, finjan ignorancia de la misma manera que en el ejercicio anterior.

In [ ]:
#Segundo Ejercicio

**Ejercicio 3:** Integra la ecuación de movimiento para el oscilador armónico, $\ddot{x} = -2x$, con $x(0)=2$, $\dot{x}(0)=0$, durante 10^4 periodos de oscilación (o sea, hasta $t_f = 10^4 \cdot 2\pi/\sqrt{2}$, usando el método de Taylor y el método de Runge-Kutta de 4o orden con paso de integración constante. Compara cómo cambia la energía (respecto al valor al tiempo cero) en función del tiempo para ambos métodos.

In [ ]:
#Tercer Ejercicio